## Passion Fruits, Object detection and localization model

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import cv2
import ast

import numpy as np 
import pandas as pd 

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as immg

import random

import torch

import torchvision
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as T
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
from tqdm.notebook import tqdm

import warnings
warnings.filterwarnings("ignore")

In [3]:
!cp 'drive/My Drive/Passion_Fruit_Dataset/Train_Images.zip' Train_Images.zip
!cp 'drive/My Drive/Passion_Fruit_Dataset/Test_Images.zip' Test_Images.zip

!unzip -q Train_Images.zip 
!unzip -q Test_Images.zip 

cp: cannot stat 'drive/My Drive/Passion_Fruit_Dataset/Train_Images.zip': No such file or directory
cp: cannot stat 'drive/My Drive/Passion_Fruit_Dataset/Test_Images.zip': No such file or directory
unzip:  cannot find or open Train_Images.zip, Train_Images.zip.zip or Train_Images.zip.ZIP.
unzip:  cannot find or open Test_Images.zip, Test_Images.zip.zip or Test_Images.zip.ZIP.


### Read the csv files

### Train

In [4]:
train_df = pd.read_csv('/content/drive/MyDrive/Passion_Fruit_Dataset/Train.csv')
train_df.head()

FileNotFoundError: ignored

In [ ]:
train_df.shape

In [ ]:
No_duplicates = train_df.drop_duplicates(subset="Image_ID")
print(No_duplicates.shape)

We have 3906 bounding boxes, from a collection of 3001 images. Some images have 1 bounding box, while others have more than 1 bounding box.

### Test

In [ ]:
test_df = pd.read_csv("/content/drive/MyDrive/Passion_Fruit_Dataset/Test.csv")
test_df.head()

In [ ]:
test_df.shape

### data preprocessing
- Calculate the xmax and ymax from the train data

In [ ]:
train_df['xmax'] = train_df['xmin']+train_df['width']
train_df['ymax'] = train_df['ymin']+train_df['height']

In [ ]:
train_df.head()

- Encode the classes from str to int

In [ ]:
classes_la = {"fruit_brownspot": 1, "fruit_healthy": 2, "fruit_woodiness":3}

train_df["class"] = train_df["class"].apply(lambda x: classes_la[x])

In [ ]:
train_df.head()

In [ ]:
train_df

- Most pretrained models reserve label 0 for the background class, therefore we encoded our labels from 1

In [ ]:
df = train_df.copy() # create a copy of the train df

### Visualization

- Let's create sample plots for our Image and the corresponding bounding box

In [ ]:
path = '/content/Train_Images/'


In [ ]:
# group by all bounding boxes (bbox)
df_grp = df.groupby(['Image_ID'])

In [ ]:
def plot_image(image_name):
    image_group = df_grp.get_group(image_name)
    bbox = image_group.loc[:,['xmin', 'ymin', 'xmax', 'ymax']]
    img = immg.imread(path+name+'.jpg')
    fig,ax = plt.subplots(figsize=(18,10))
    ax.imshow(img,cmap='binary')
    for i in range(len(bbox)):
        box = bbox.iloc[i].values
        print(bbox)
        x,y,w,h = box[0], box[1], box[2]-box[0], box[3]-box[1]
        rect = matplotlib.patches.Rectangle((x,y),w,h,linewidth=1,edgecolor='r',facecolor='none',)
        ax.text(*box[:2], image_group["class"].values, verticalalignment='top', color='white', fontsize=13, weight='bold')
        ax.add_patch(rect)
    plt.show()

In [ ]:
name = df.Image_ID.unique()[0]
plot_image(name)

In [ ]:
name = df.Image_ID.unique()[700]
plot_image(name)

In [ ]:
name = df.Image_ID.unique()[67]
plot_image(name)

#### Create a custom dataset

- We'll create a custom dataset to load our images into PyTorch object detector.

- Sample https://pytorch.org/tutorials/intermediate/torchvision_tutorial.html

In [ ]:
class PassionFruit(object):
    def __init__(self, df, IMG_DIR, transforms): 
        self.df = df
        self.img_dir = IMG_DIR
        self.image_ids = self.df['Image_ID'].unique().tolist()
        self.transforms = transforms
        
    def __len__(self):
        return len(self.image_ids)
    
    def __getitem__(self, idx):
        image_id = self.image_ids[idx]
        image_values = self.df[self.df['Image_ID'] == image_id]
        image = cv2.imread(self.img_dir+image_id+".jpg",cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0
        
        boxes = image_values[['xmin', 'ymin', 'xmax', 'ymax']].to_numpy()
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        
        labels = image_values["class"].values
        labels = torch.tensor(labels)
        
        target = {}
        target['boxes'] = boxes
        target['labels'] = labels
        target['image_id'] = torch.tensor([idx])
        target['area'] = torch.as_tensor(area, dtype=torch.float32)
        target['iscrowd'] = torch.zeros(len(classes_la), dtype=torch.int64)

        if self.transforms:
            sample = {
                'image': image,
                'bboxes': target['boxes'],
                'labels': labels
            }
        
            sample = self.transforms(**sample)
            image = sample['image']
            
            target['boxes'] = torch.stack(tuple(map(torch.tensor, zip(*sample['bboxes'])))).permute(1, 0)

        return torch.tensor(image), target, image_id

- Define the image transformations. We'll use albumentations package: https://albumentations.ai/

In [ ]:
pip install -U albumentations


In [ ]:
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
def get_train_transform():
    return A.Compose([
        A.Flip(0.5),
        A.RandomBrightnessContrast(p=0.2),
        A.RandomGamma(p=1),
        A.ShiftScaleRotate(p=0.5),
        ToTensorV2(p=1.0)
    ], bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})

def get_valid_transform():
    return A.Compose([
        ToTensorV2(p=1.0)
    ], bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})

- load passion fruit dataset and perform the transormations

In [ ]:
passion_dataset = PassionFruit(df, path, get_train_transform())

- Check if the custom dataset object created ealier works

In [ ]:
img, tar, _ = passion_dataset[random.randint(0,1000)]
bbox = tar['boxes']
fig,ax = plt.subplots(figsize=(18,10))
ax.imshow(img.permute(1,2,0).cpu().numpy())
for l in tar["labels"].tolist():
    classes_la = {1:"fruit_brownspot",  2:"fruit_healthy", 3:"fruit_woodiness"}
    l = classes_la[l]
    for i in range(len(bbox)):
        box = bbox[i]
        x,y,w,h = box[0], box[1], box[2]-box[0], box[3]-box[1]
        rect = matplotlib.patches.Rectangle((x,y),w,h,linewidth=2,edgecolor='r',facecolor='none',)
        ax.text(*box[:2], l, verticalalignment='top', color='red', fontsize=13, weight='bold')
        ax.add_patch(rect)
    plt.show()

#### Split data into training and test

In [ ]:
image_ids = df['Image_ID'].unique()
valid_ids = image_ids[-665:]
train_ids = image_ids[:-665]
valid_df = df[df['Image_ID'].isin(valid_ids)]
train_df = df[df['Image_ID'].isin(train_ids)]
train_df.shape,valid_df.shape

### Data Loader

- PyTorch uses dataloader module to make the dataset iterable. We'll pass both the train and val set to the PyTorch DataLoader

In [ ]:
def collate_fn(batch):
    return tuple(zip(*batch))

train_dataset = PassionFruit(df, path, get_train_transform())
valid_dataset = PassionFruit(df, path, get_valid_transform())

# split the dataset in train and test set
indices = torch.randperm(len(train_dataset)).tolist()

train_data_loader = DataLoader(
    train_dataset,
    batch_size=8,
    shuffle=False,
    num_workers=4,
    collate_fn=collate_fn
)

valid_data_loader = DataLoader(
    valid_dataset,
    batch_size=8,
    shuffle=False,
    num_workers=4,
    collate_fn=collate_fn
)

- Most pretrained models are trained with a background class, we'll include it in our model, so in that case our number of classes will be 4

In [ ]:
num_classes = 4 # + background

# load a model; pre-trained on COCO
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)


# get number of input features for the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features

# replace the pre-trained head with a new one
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

- Train either on GPU if available else CPU 

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
model.to(device)
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.009, momentum=0.9, weight_decay=0.0005)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

In [ ]:
num_epochs = 5

#### Training and evaluation

In [ ]:
# import sys
# best_epoch = 0
# min_loss = sys.maxsize

# for epoch in range(num_epochs):
#     tk = tqdm(train_data_loader)
#     model.train();
#     for images, targets, image_ids in tk:
#         images = list(image.to(device) for image in images)
#         targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

#         loss_dict = model(images, targets)

#         losses = sum(loss for loss in loss_dict.values())
#         loss_value = losses.item()

#         optimizer.zero_grad()
#         losses.backward()
#         optimizer.step()
        
#         tk.set_postfix(train_loss=loss_value)
#     tk.close()
    
#     # update the learning rate
#     if lr_scheduler is not None:
#         lr_scheduler.step()
    
#     print(f"Epoch #{epoch} loss: {loss_value}") 
        
#     #validation 
#     model.eval();
#     with torch.no_grad():
#         tk = tqdm(valid_data_loader)
#         for images, targets, image_ids in tk:
        
#             images = list(image.to(device) for image in images)
#             targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
#             val_output = model(images)
#             val_output = [{k: v.to('cpu') for k, v in t.items()} for t in val_output]
#             IOU = []
#             for j in range(len(val_output)):
#                 a,b = val_output[j]['boxes'].cpu().detach(), targets[j]['boxes'].cpu().detach()
#                 chk = torchvision.ops.box_iou(a,b)
#                 res = np.nanmean(chk.sum(axis=1)/(chk>0).sum(axis=1))
#                 IOU.append(res)
#             tk.set_postfix(IoU=np.mean(IOU))
#         tk.close()

In [ ]:
model.load_state_dict(torch.load('/content/drive/MyDrive/Data/passion_fruit_model_V2.pt', map_location=torch.device('cpu')))

##### Sample evaluation on validation dataset image

In [ ]:
img,target,_ = valid_dataset[5]
# put the model in evaluation mode
model.eval()
with torch.no_grad():
    prediction = model([img.to(device)])[0]
    
print('predicted #boxes: ', len(prediction['boxes']))
print('real #boxes: ', len(target['boxes']))

In [ ]:
prediction

## ground truth box for the sample test

In [ ]:
bbox = target['boxes'].numpy()
fig,ax = plt.subplots(1,figsize=(18,10))
ax.imshow(img.permute(1,2,0).cpu().numpy())
for l in target["labels"]:
    for i in range(len(bbox)):
        box = bbox[i]
        x,y,w,h = box[0], box[1], box[2]-box[0], box[3]-box[1]
        rect = matplotlib.patches.Rectangle((x,y),w,h,linewidth=2,edgecolor='r',facecolor='none',)
        ax.text(*box[:2], l.tolist(), verticalalignment='top', color='red', fontsize=13, weight='bold')
        ax.add_patch(rect)
    plt.show()

## Predicted box for the sample Image

In [ ]:
def plot_valid(img,prediction,nms=True,detect_thresh=0.5):
    fig,ax = plt.subplots(figsize=(18,10))
    val_img = img.permute(1,2,0).cpu().numpy()
    ax.imshow(val_img)
    val_scores = prediction['scores'].cpu().detach().numpy()
    bbox = prediction['boxes'].cpu().detach().numpy()
    for l in target["labels"]:
        for i in range(len(bbox)):
            if val_scores[i]>=detect_thresh:
                box = bbox[i]
                x,y,w,h = box[0], box[1], box[2]-box[0], box[3]-box[1]
                rect = matplotlib.patches.Rectangle((x,y),w,h,linewidth=2 ,edgecolor='r',facecolor='none',)
                ax.text(*box[:2], "class " + str(l.tolist()) + " score {0:.3f}".format(val_scores[i]), verticalalignment='top', color='white', fontsize=12, weight='bold')
                ax.add_patch(rect)
        plt.show()

In [ ]:

plot_valid(img,prediction)

#### Load the submission images IDs

- We can load them from the test.csv

In [ ]:
submission = pd.read_csv("/content/drive/MyDrive/Passion_Fruit_Dataset/Test.csv")


## Create a custom dataset for the test set

In [ ]:
class TestDataset(object):
    def __init__(self, df, IMG_DIR, transforms):        
        self.df = df
        self.img_dir = IMG_DIR
        self.transforms = transforms
        self.image_ids = self.df['Image_ID'].tolist()
        
    def __len__(self):
        return len(self.image_ids)
    
    def __getitem__(self, idx):        
        image_id = self.image_ids[idx]
        image = cv2.imread(self.img_dir+image_id+".jpg",cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        
        if self.transforms:
            sample = {
                'image': image,
            }
            sample = self.transforms(**sample)
            image = sample['image']
        return image, image_id

### Apply transformation

In [ ]:
def get_test_transform(IMG_SIZE=(512,512)):
    return A.Compose([
         A.Normalize(mean=(0, 0, 0), std=(1, 1, 1), max_pixel_value=255.0, p=1.0),
        A.Resize(*IMG_SIZE),
        ToTensorV2(p=1.0)
    ])

In [ ]:
# test_img_dir = '/content/drive/MyDrive/Try/images/test/'
test_img_dir = "/content/Test_Images/"

- The training images were of size 515 x 512, resize the test images to the same dimensions.

In [ ]:
IMG_SIZE = (512,512)

In [ ]:
test_dataset = TestDataset(submission, test_img_dir ,get_test_transform())

### Predicting on testset,  create submission

In [ ]:
results = []
for j in range(submission.shape[0]):
    
    img,_ = test_dataset[j]
    img = img.unsqueeze_(0)
    # put the model in evaluation mode
    model.eval()

    with torch.no_grad():
        prediction = model([img.to(device)][0])
        aa = zip(prediction[0]["boxes"].tolist(), prediction[0]["labels"].tolist(), prediction[0]["scores"].tolist())
       
        for item in list(aa):
            row_dict = {}
            row_dict["Image_ID"] = _
            row_dict["boxes"] = item[0]
            row_dict["labels"] = item[1]
            row_dict["confidence"] = item[2]
            results.append(row_dict)
sub_df = pd.DataFrame(results)

In [ ]:
sub_df["ymin"] = sub_df["boxes"].apply(lambda x: x[1])
sub_df["xmin"] = sub_df["boxes"].apply(lambda x: x[0])
sub_df["ymax"] = sub_df["boxes"].apply(lambda x: x[3])
sub_df["xmax"]=  sub_df["boxes"].apply(lambda x: x[2])

In [ ]:
classes_la = {0:"Background", 1:"fruit_brownspot", 2:"fruit_healthy", 3:"fruit_woodiness"}
sub_df["labels"] = sub_df["labels"].apply(lambda x: classes_la[x])

In [ ]:
sub_df.drop(["boxes"], axis=1, inplace=True)

In [ ]:
sub_df.rename(columns={"labels":"class"}, inplace=True)

In [ ]:
sub_df.head()

In [ ]:
sub_df.shape

In [ ]:
sub_df.to_csv("/content/drive/MyDrive/Try/images/baseline_solution5.csv", index=False)